## create_datasets를 먼저 돌리고 실행

In [1]:
import pandas as pd

## Size 요소 가져오기

In [13]:
market_cap_data = pd.read_hdf('assets.h5', 'finance_datareader/prices')['rank']          

In [14]:
market_cap_data

date        ticker
1995-05-02  000020    153
            000040    292
            000050    144
            000060     70
            000070     49
                     ... 
2021-05-13  365550    213
            375500    112
            378850    637
            900140    385
            950210    142
Name: rank, Length: 3802553, dtype: int16

In [15]:
total_asset = pd.read_csv('IFRS/TotalAssets.csv', encoding='CP949')

In [16]:
# 종목코드를 6자리로 맞춰줌 -> 파이썬 고질병 '000660'을 숫자로 자동 인식하면 앞의 000을 날려버림 이거를 string으로 변경해서 복구
ticker = total_asset['거래소코드'].apply("{0:0>6}".format)

In [17]:
# 금융업 제외한 재무제표랑 market cap 데이터랑 겹치는 종목 추출
intersect_ticker = market_cap_data.unstack('date').index.intersection(ticker)

In [18]:
size_factor = (market_cap_data
                .unstack('date')
                .loc[intersect_ticker,:].stack('date')
                .to_frame()
                .reorder_levels(['date','ticker'])
                .sort_index())

In [27]:
size_factor.columns = ['rank']
size_factor = size_factor.astype('int32')

In [28]:
size_factor

rank
date       ticker      
1995-05-02 000020   153
           000040   292
           000050   144
           000070    49
           000080    67
...                 ...
2021-05-13 363280   208
           375500   112
           378850   637
           900140   385
           950210   142

[3539813 rows x 1 columns]

## B/M Book to Market Ratio 구하기

In [150]:
def get_ifrs_data(data_path: str):
    data = pd.read_csv(data_path, encoding='CP949')
    data.fillna(0, inplace=True)
    data['거래소코드'] = data['거래소코드'].apply("{0:0>6}".format)
    data['회계년도'] = pd.to_datetime(data['회계년도'])
    data = data.set_index(['회계년도','거래소코드']).sort_index()
#     data = data.index.levels[0].resample('A').last()
#     data.index = data.index.set_levels(data.index.levels[0].to_period('M'), level=0)
    return data.copy()

In [151]:
common_stock_capital = get_ifrs_data('IFRS/CommonStock.csv')

In [152]:
common_stock_capital.unstack()

회사명  보통주자본금(천원)  보통주자본금(IFRS)(천원)
회계년도       거래소코드                                                
1981-02-01 004980          성신양회(주)   5303131.0               0.0
1981-03-01 002700          신일전자(주)   4350000.0               0.0
           009830         한화솔루션(주)  13277500.0               0.0
1981-06-01 001550            (주)조비   1000000.0               0.0
1981-07-01 001070          대한방직(주)   3000000.0               0.0
...                            ...         ...               ...
2020-12-01 352820           (주)하이브  17811880.0        17811880.0
           353200          대덕전자(주)  24708462.0        24708462.0
           361610  에스케이아이이테크놀로지(주)  62741592.0        62741592.0
           363280        (주)티와이홀딩스  18750451.0        18750451.0
           900140      엘브이엠씨홀딩스(주)         0.0               0.0

[20876 rows x 3 columns]

In [120]:
capital_surplus = get_ifrs_data('IFRS/CapitalSurplus.csv')

In [121]:
capital_surplus

회사명  자본잉여금(*)(천원)  자본잉여금(*)(IFRS)(천원)
회계년도    거래소코드                                                    
1981-02 004980          성신양회(주)  1.584780e+05        0.000000e+00
1981-03 002700          신일전자(주)  1.457700e+04        0.000000e+00
        009830         한화솔루션(주)  3.591559e+06        0.000000e+00
1981-06 001550            (주)조비  7.500000e+04        0.000000e+00
1981-07 001070          대한방직(주)  6.167838e+06        0.000000e+00
...                         ...           ...                 ...
2020-12 352820           (주)하이브  1.187589e+09        1.187589e+09
        353200          대덕전자(주)  5.496102e+08        5.496102e+08
        361610  에스케이아이이테크놀로지(주)  1.018150e+09        1.018150e+09
        363280        (주)티와이홀딩스  5.285961e+08        5.285961e+08
        900140      엘브이엠씨홀딩스(주)  0.000000e+00        0.000000e+00

[20876 rows x 3 columns]

In [122]:
retained_earnings = get_ifrs_data('IFRS/RetainedEarnings.csv')

In [123]:
retained_earnings

회사명  이익잉여금(*)(천원)  이익잉여금(결손금)(*)(IFRS)(천원)
회계년도    거래소코드                                                         
1981-02 004980          성신양회(주)      844948.0                      0.0
1981-03 002700          신일전자(주)           0.0                      0.0
        009830         한화솔루션(주)      311872.0                      0.0
1981-06 001550            (주)조비      241598.0                      0.0
1981-07 001070          대한방직(주)           0.0                      0.0
...                         ...           ...                      ...
2020-12 352820           (주)하이브    -2959997.0               -2959997.0
        353200          대덕전자(주)    67335905.0               67335905.0
        361610  에스케이아이이테크놀로지(주)   154084273.0              154084273.0
        363280        (주)티와이홀딩스    -2390485.0               -2390485.0
        900140      엘브이엠씨홀딩스(주)           0.0                      0.0

[20876 rows x 3 columns]

In [124]:
deferred_tax_liabilities = get_ifrs_data('IFRS/DeferredTaxLiabilities.csv')

In [125]:
deferred_tax_liabilities

회사명  이연법인세부채(천원)  이연법인세부채(IFRS)(천원)
회계년도    거래소코드                                                  
1981-02 004980          성신양회(주)          0.0                0.0
1981-03 002700          신일전자(주)          0.0                0.0
        009830         한화솔루션(주)          0.0                0.0
1981-06 001550            (주)조비          0.0                0.0
1981-07 001070          대한방직(주)          0.0                0.0
...                         ...          ...                ...
2020-12 352820           (주)하이브          0.0          5407005.0
        353200          대덕전자(주)          0.0                0.0
        361610  에스케이아이이테크놀로지(주)          0.0                0.0
        363280        (주)티와이홀딩스          0.0         12464108.0
        900140      엘브이엠씨홀딩스(주)          0.0                0.0

[20876 rows x 3 columns]

In [129]:
tmp = deferred_tax_liabilities.loc[idx['1981',:], :]['회사명']